In [38]:
import tensorflow as tf
from tensorflow import keras
import spacy
import os
import json
import codecs
import pandas as pd
import numpy as np
import random

nlp = spacy.load("en_core_web_sm")
print(tf.__version__)

2.9.2


In [ ]:
query = nlp('When was Facebook founded')
query.ents

(Facebook,)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Change working directory to be current folder
os.chdir('/content/gdrive/My Drive/MyColab/PLP')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 bigram_vectorizer.pkl		   SBert_embedder.pkl
 ch21.pkl			   Small_talk_Intent.csv
 chatbot_intent			   Small_talk_Intent.xlsx
 clr_svm.pkl			  'spi_index_labelled vFinal.csv'
 corpus_embeddings.pkl		  'spi_index_labelled vFinal.db'
 enc_class.pkl			   terms_questions.csv
 gqlmodel.h5			   textsql.csv
 GQLquestion.pkl		   tfidf_matrix.pkl
 investopedia_corpus.csv	   tfidf.pkl
 investopedia_terms_question.csv


In [ ]:
#random pick 3000 questions from sql training data
sql_questions_df = pd.read_csv('textsql.csv')
sql_questions = random.choices(list(sql_questions_df['query']),k=3000)

#SQL Query Result Text Generation

Apply rule based method based on the spacy token pos tag to decide the generated reply.

In [ ]:
#Rule 1 for SQL query text generation
#if the submitted query with any interrogative word, the extraction of the query to perform text generation is easier and more straightforward
ques_ent = nlp('What is the count of Environmental Statistics?')
from spacy import displacy
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

# The important words always come with "determiners"
#text generation can reply as: The count of Environmental statistics is...

In [ ]:
#Rule 2 for SQL query text generation
#if the submitted query is not a query but a action, eg:
ques_ent = nlp('list me the business or establishment census of Tunisia after 2014?')
from spacy import displacy
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

# The important words are "Verb" + "Pron" or "Verb" + "Pron" or "Intj" + "Verb" + "Pron"
#text generation can reply as: The count of Environmental statistics is...

In [ ]:
#Rule 3 for SQL query text generation
#if the submitted query is started with noun
ques_ent = nlp('Countries having SPI Overall Score between 54 and 62 in 2019')
from spacy import displacy
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#Get whole sentence directly

In [ ]:
#Rule 4 for SQL query text generation
#if the submitted query is started with first entity detected
ques_ent = nlp('Data Products Score and Download Options Score of Sao Tome and Principe since 2010')
from spacy import displacy
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

print(ques_ent.ents[0].text)
print(ques_ent[:len(ques_ent.ents[0])].text)
#Get whole sentence directly

Data Products Score
Data Products Score


In [ ]:
#Rule others for SQL query text generation
#if the submitted query without any interrogative word or determiners
ques_ent = nlp('Responsible Consumption and Production of Turkey in 2011.')
from spacy import displacy
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#it is difficult to set rule for the random terms unless there are dictionaries to map the terms correctly
#The leftover sentence will be ignored and provide the sql query result directly.

In [ ]:
textGeneratedList = []
skipTextList = []
for question in sql_questions:
  ques_ent = nlp(question)
  start = 0
  sentList = []
  #rule 1
  for sent in ques_ent.sents:
    for token in sent:
      if not start and (token.pos_ == 'PRON' and token.head.pos_ == 'AUX'):
        start = 1
      elif start==1 and (token.pos_ == 'AUX'):
        start = 2
      elif start==2 and not token.is_sent_end and not token.is_punct:
        sentList.append(token.text)
  if len(sentList)==0:
    #rule 2
    start = 0
    for sent in ques_ent.sents:
      for token in sent:
        if not start and (token.pos_ == 'VERB' and token.dep_ == 'ROOT'):
          start = 1
        elif start==1 and (token.pos_ == 'DET' or token.pos_ == 'PRON'):
          start = 2
          if token.pos_ == 'DET':
            sentList.append(token.text)
        elif start==2 and not token.is_sent_end and not token.is_punct:
          sentList.append(token.text)
    if len(sentList)==0:
      start = 0
      #rule 3
      for sent in ques_ent.sents:
        for token in sent:
          if token.pos_ == 'NOUN':
            start = 1
          break
        break
      if start:
        if ques_ent.doc[-1].is_punct:
          textGeneratedList.append(ques_ent.doc[:-1].text)
        else:
          textGeneratedList.append(ques_ent.doc.text)
      else:
        #rule 4
        entities = ques_ent.ents
        if len(entities)>0:
          length = len(entities[0])
          if entities[0].text==ques_ent[:length].text:
            if ques_ent.doc[-1].is_punct:
              textGeneratedList.append(ques_ent.doc[:-1].text)
            else:
              textGeneratedList.append(ques_ent.doc.text)
          else:
            #rule others
            skipTextList.append(question)
        else:
          #rule others
          skipTextList.append(question)
    else:
      textGeneratedList.append(' '.join(sentList))
  else:
    textGeneratedList.append(' '.join(sentList))
print(len(textGeneratedList))
print(len(skipTextList))

2406
594


In [ ]:
for data in textGeneratedList:
  print(data)

Countries having Terms of Use Score between 0.1 and 0.3 in 2005
maximum Data use by international organizations by region
Countries having Download Options Score between 0.2 and 0.5 in 2017
average of Standards and Methods in Latin America & Caribbean
average of income in 2005
a day in 2013
population of Nepal after 2016
Peace and Justice Strong Institutions of North Macedonia after 2013
List of No Poverty of Guatemala after 2012
Countries having Non_Proprietary format Score between 0.0 and 0.1
the Countries having Data Sources Score between 38 and 45 in
the Data Use Score and ODIN Open Data Openness Score of Kenya since
Business process of Korea, Dem. People’s Rep. before 2008
Countries having ODIN Open Data Openness Score between 0.0 and 0.1 in 2012
List of Countries having Data Infrastructure Score between 47 and 50 in 2005
Countries having Metadata Available Score between 0.1 and 0.2
the Institutional Statistics of Portugal after 2004
Business/establishment census and Population & 

In [ ]:
skipTextList

['average Compilation of government finance statistics by region',
 'minimum Standards and Methods by region?',
 'Machine Readability Score of Singapore in 2008?',
 'Online access and Labor Force Survey of Kiribati since 2014.',
 'Reduced Inequality of Norway before 2013.',
 'Top 35 countries by Social Statistics.',
 'Good Health and Well-being of Denmark before 2015?',
 'List Top 54 countries by Sustainable Cities and Communities.',
 'Top 34 countries by Classification of status of employment?',
 'Data Sources Score of West Bank and Gaza in 2015?',
 'List Reduced Inequality of Trinidad and Tobago after 2007.',
 'maximum Compilation of government finance statistics by region.',
 'average Peace and Justice Strong Institutions by region.',
 "Machine Readability Score of Cote d'Ivoire in 2011.",
 'Top 12 countries by SDDS/e-GDDS subscription?',
 'count Classification of national industry by region.',
 'Top 36 countries by Agriculture census.',
 'Top 29 countries by Data Releases',
 'Decen

#GQL Query Result Text Generation

Apply rule based method based on the spacy entities to decide the generated reply.

In [ ]:
import pickle
with open('GQLquestion.pkl','rb') as f:
  gql_questions = pickle.load(f)
#random pick 3000 questions from gql training data
gql_questions = random.choices(gql_questions,k=3000)

In [ ]:
gql_questions

['When was Harbour Group founded?',
 'What is the name of the investment banking subsidiary of FTI Consulting?',
 "Where was Brendan O'Brien based?",
 'When was the Meltdown flaw reported?',
 'Where is Seaspan Shipyards?',
 'Who is the official spokesperson for Fantag?',
 "Based on the Fund's NAV as of what date?",
 "What is Canada's most populous province?",
 "Who is Olukai's co-founder?",
 'Hitachi Vantara is a wholly owned subsidiary of what company?',
 'Who was the imprisoned leader of the outlawed Kurdistan Workers’ Party?',
 'Who is the senior partner at Control Risks?',
 'Who is the founder of Shegerian & Associates?',
 'Who is the Chairman and Chief Executive Officer of Phoenix Marketing International?',
 'Atchuthan Srirangan is a senior research analyst at what company?',
 'Who vs. Bradford City?',
 'Who is the leader of South Korea?',
 'What is the biggest foreign lender to Reliance Communications Ltd (RCom)?',
 'Who was the Chairman of Billabong?',
 'What was the name of the

In [ ]:
from spacy import displacy
"""
Rule 1: Handling on the "Who" interrogative word.
If the query fulfills the condition 1 and 2, replace token "Who" with the answer.
If the condition 3 is met, remove token "Who" and any AUX token after interrogative word and place the answer right after the VERB token or ADP/ADV token.
"""

#condition 1
ques_ent = nlp('Who was HSH Chairman?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 2
ques_ent = nlp('Who owns the property firm behind Ayala Center Cebu?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 3
ques_ent = nlp('Who did Alan Baldwin work for?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })


In [ ]:
"""
Rule 2: Handling on the "When" interrogative word.
If the query fulfills the condition 1, remove token "When" and any AUX token after interrogative word and place the answer at the end of the statement.
If the condition 2 is met, AUX token such as "is" "are" "was" "were" are moved and placed before the "VERB token" and the answer is still placed at the end of the statement.
"""

#condition 1
ques_ent = nlp('When did Zuckerberg start his company?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 2
ques_ent = nlp('When was McDonald Hopkins founded?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

In [ ]:
"""
Rule 3: Handling on the "What" or "Which" interrogative word.
The handling of the rule is similar to the "Who" interrogative word. Additonal condition 4 is considered to ignore any NOUN or PRON token after the interrogative word.
If the query fulfills the condition 1 and 2, replace token "What" with the answer.
If the condition 3 is met, remove token "What" and any AUX token after interrogative word and place the answer right after the VERB token or ADP/ADV token.
"""

#condition 1
ques_ent = nlp('What is the name of the anchor who officially co-anchored Today?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 2
ques_ent = nlp("What adds true organic, massive scale and topical coverage to Maven's core strategy of aggregating independent, niche, passion-based content channels?")
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 3
ques_ent = nlp('What did Twitter decline to comment on?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 4
ques_ent = nlp('What company did Celgene Corp buy on Sunday?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })


In [ ]:
"""
Rule 4: Handling on the "Where" interrogative word.
The handling is same as the interrogative word "When"
"""

#condition 1
ques_ent = nlp('Where was the capital of Belize?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

#condition 2
ques_ent = nlp('Where was David Ingram based?')
# visualize the dependencies.
displacy.render(ques_ent.sents, style='dep', jupyter=True, options = { 'distance':80 })

In [ ]:
a='i am YOUR boy'
a.capitalize()

'I am your boy'

In [ ]:
skipTextList = []
textGeneratedList = []
for question in gql_questions:
  if question.startswith('In w'):
    question = question[3].upper() + question[4:]
  ques_ent = nlp(question)
  start = 0
  rule = ''
  sentList = []
  tempKey = ''
  tempList = []
  for sent in ques_ent.sents:
    for token in sent:

      if not start and token.text=='Who':
        start = 1
        rule = token.text
      #rule 1
      elif rule=='Who':
        if start==1 and (token.pos_ == 'AUX') and len(tempList)==0 and token.text in ['is','are','was','were','has','have']:
          start = 2
          tempKey = f"XXX {token.text}"
          sentList.append(tempKey)
        elif start==1 and (token.pos_ == 'VERB') and len(tempList)==0 :
          start = 2
          tempKey = f"XXX {token.text}"
          sentList.append(tempKey)
        elif start==1 and tempKey=='' and (token.pos_ == 'PROPN' or token.pos_ == 'CCONJ' or token.pos_ == 'NOUN' or token.pos_ == 'VERB'):
          tempList.append(token.text)
        elif start==1 and len(tempList) and ((token.pos_ == 'ADP' and token.head.pos_=='VERB') or (token.pos_ == 'ADV' and token.head.pos_=='VERB') or token.head.pos_!='VERB'):
          if token.head.pos_=='VERB':
            tempList.append(f"{token.text} XXX")
          else:
            tempList.append(f"XXX {token.text}")
          start = 2
          sentList.extend(tempList)
        elif start==2 and not token.is_sent_end:
          sentList.append(token.text)

      #rule 2
      elif not start and token.text=='When':
        start = 1
        rule = token.text
      elif rule=='When':
        if start==1 and (token.pos_ == 'AUX') and token.text in ['is','are','was','were','did','will']:
          start = 2
          if token.text in ['was','were','is','are']:
            tempKey = token.text
        elif start==2 and tempKey!='' and not token.is_sent_end and not token.is_punct and token.pos_ == 'VERB' and token.dep_=='ROOT':
          sentList.append(f"{tempKey} {token.text}")
          tempKey = ''
        elif start==2 and not token.is_sent_end:
          sentList.append(token.text)
        elif token.is_sent_end and token.is_punct:
          sentList.append(f"in XXX")

      #rule 3
      elif not start and (token.text=='What' or token.text=='Which'):
        start = 1
        rule = token.text
      elif rule=='What' or rule=='Which':
        if start==1 and (token.pos_ == 'NOUN') and len(tempList)==0:
          continue
        elif start==1 and (token.pos_ == 'AUX') and len(tempList)==0 and token.text in ['did','does','do']:
          continue
        elif start==1 and tempKey=='' and len(tempList)==0 and (token.pos_ == 'AUX') and token.text in ['is','are','was','were','will']:
          start = 2
          tempKey = f"XXX {token.text}"
          sentList.append(tempKey)
        elif start==1 and tempKey=='' and (token.pos_ == 'VERB') and len(tempList)==0:
          start = 2
          tempKey = f"XXX {token.text}"
          sentList.append(tempKey)
        elif start==1 and tempKey=='' and ( token.pos_ == 'AUX' or token.pos_ == 'PROPN' or token.pos_ == 'CCONJ' or token.pos_ == 'NOUN' or token.pos_ == 'VERB'):
          tempList.append(token.text)
        elif start==1 and len(tempList) and ((token.pos_ == 'ADP' and token.head.pos_=='VERB') or (token.pos_ == 'ADV' and token.head.pos_=='VERB') or token.head.pos_!='VERB'):
          if token.head.pos_=='VERB':
            tempList.append(f"{token.text} XXX")
          else:
            tempList.append(f"XXX {token.text}")
          start = 2
          sentList.extend(tempList)
        elif start==2 and not token.is_sent_end:
          sentList.append(token.text)

      #rule 4
      elif not start and token.text=='Where':
        start = 1
        rule = token.text
      elif rule=='Where':
        if start==1 and (token.pos_ == 'AUX') and token.text in ['is','are','was','were','did','will']:
          start = 2
          if token.text in ['was','were','is','are']:
            tempKey = token.text
        elif start==2 and tempKey!='' and not token.is_sent_end and not token.is_punct and token.pos_ == 'VERB' and token.dep_=='ROOT':
          sentList.append(f"{tempKey} {token.text}")
          tempKey = ''
        elif start==2 and not token.is_sent_end:
          sentList.append(token.text)
        elif token.is_sent_end and token.is_punct:
          sentList.append(f"in XXX")

  if len(sentList)!=0 and ((rule=='Who' and start==2) or (rule=='When' and start==2) or ((rule=='What' or rule=='Which') and start==2) or (rule=='Where' and start==2)):
    textGeneratedList.append(' '.join(sentList))
  else:
    skipTextList.append(ques_ent.text)

print(len(textGeneratedList))
print(len(skipTextList))

2426
574


In [ ]:
for data in textGeneratedList:
  print(data)

Harbour Group was founded in XXX
XXX is the name of the investment banking subsidiary of FTI Consulting
Brendan O'Brien based in XXX
the Meltdown flaw was reported in XXX
Seaspan Shipyards in XXX
XXX is the official spokesperson for Fantag
XXX is Canada 's most populous province
XXX is Olukai 's co - founder
XXX was the imprisoned leader of the outlawed Kurdistan Workers ’ Party
XXX is the senior partner at Control Risks
XXX is the founder of Shegerian & Associates
XXX is the Chairman and Chief Executive Officer of Phoenix Marketing International
Bradford City XXX ?
XXX is the leader of South Korea
XXX is the biggest foreign lender to Reliance Communications Ltd ( RCom )
XXX was the Chairman of Billabong
XXX was the name of the 7GENx founder
XXX is the name of the fund that EnCap Flatrock is making commitments to
Vision Zero originally was developed in XXX
ICMA - RC was founded in XXX
XXX reversed a decision to locate the insolvency in Germany
XXX is Brian Sandoval
XXX was one of the s

In [ ]:
skipTextList

["Based on the Fund's NAV as of what date?",
 'Hitachi Vantara is a wholly owned subsidiary of what company?',
 'Atchuthan Srirangan is a senior research analyst at what company?',
 'Hurricane Maria lingered in what area in the fourth quarter?',
 'On what day does Alabama host Auburn?',
 'Niki Blasina explains the four main proxy conflicts between what two nations?',
 'Dominic Chu and Gina Francolla contributed to this report?',
 'The most recent Robertshaw acquisition was which company?',
 "Lionel Messi and Luis Suarez visit children's hospital in what city?",
 'Viorica Dancila is a member of what body?',
 'Jonathan Stempel is based in what city?',
 'Nate Raymond is based in Boston, where is he based?',
 'Nigeria is starting flights to evacuate thousands of its citizens from what country?',
 'Where does a United Airlines passenger get boarding documents?',
 'Alexandra Schwarz-Goerlich and Francois Murphy were in Vienna where?',
 'Alexandra Schwarz-Goerlich and Francois Murphy were in 